In [1]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
## Import Libraries

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import cv2

In [3]:
## Load and Preprocess FER2013 Dataset:


# Assuming you have FER2013.csv downloaded and stored locally
df = pd.read_csv('fer2013.csv')

# Extract pixels and labels from the dataset
pixels = df['pixels'].tolist()
labels = df['emotion'].astype('str').tolist()
# print(labels)

# Convert pixels to numpy arrays and reshape for image dimensions
images = np.array([np.fromstring(pixel, dtype=int, sep=' ').reshape(48, 48, 1) for pixel in pixels])
labels = np.array(labels)

# Normalize pixel values to the range [0, 1]
images = images / 255.0

# Convert emotion labels to one-hot encoded format
label_encoder = LabelEncoder()
labels = tf.keras.utils.to_categorical(label_encoder.fit_transform(labels), num_classes=7)


# Assuming you have FER2013.csv downloaded and stored locally
df = pd.read_csv('./fer2013.csv')

# Extract pixels and labels from the dataset
pixels = df['pixels'].tolist()
labels = df['emotion'].astype('str').tolist()
# print(labels)

# Convert pixels to numpy arrays and reshape for image dimensions
images = np.array([np.fromstring(pixel, dtype=int, sep=' ').reshape(48, 48, 1) for pixel in pixels])
labels = np.array(labels)

# Normalize pixel values to the range [0, 1]
images = images / 255.0

# Convert emotion labels to one-hot encoded format
label_encoder = LabelEncoder()
labels = tf.keras.utils.to_categorical(label_encoder.fit_transform(labels), num_classes=7)


In [4]:
## Split the dataset

train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)


In [5]:
## Build and Train the Model

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(7, activation='softmax')  # 7 output classes for different emotions
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, validation_split=0.2)





Epoch 1/10


718/718 [==============================] - 22s 28ms/step - loss: 1.6615 - accuracy: 0.3401 - val_loss: 1.5211 - val_accuracy: 0.4086
Epoch 2/10
130/718 [====>.........................] - ETA: 14s - loss: 1.4706 - accuracy: 0.4300

KeyboardInterrupt: 

In [ ]:
## Evaluate the model

test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test Accuracy: {test_acc}')

In [6]:
## Real-Time Video Emotion Detection:

cap = cv2.VideoCapture(0)  # Use 0 for webcam

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess frame (resize, normalize, convert to grayscale, etc.)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     frame = cv2.resize(frame, (48, 48))
#     frame = frame.reshape((1, 48, 48, 1))
    frame = cv2.resize(frame, (240, 240))
    frame = frame.reshape((1, 240, 240, 1))
    frame = frame / 255.0
#     frame = cv2.resize(frame, (48, 48))
#     gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     normalized_frame = gray_frame.reshape((1, 48, 3)).astype('float32') / 255.0  # Adjust the shape here


    # Run inference
    predictions = model.predict(frame)

    # Get the predicted emotion label
    predicted_label = label_encoder.inverse_transform([np.argmax(predictions)])

    print(f"Predicted Emotion: {predicted_label}")

    
    print(frame.shape)
    frame = np.squeeze(frame)
    print(frame.shape)
#     frame = np.squeeze(frame).astype(np.uint8)
    
    # Display results
    cv2.putText(frame, predicted_label[0], (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Emotion Detection', frame)

    if cv2.waitKey(1) & 0xFF == 27:  # Press 'Esc' to exit
        break

cap.release()
cv2.destroyAllWindows()


ValueError: in user code:

    File "C:\InstalledUtilities\Anaconda\lib\site-packages\keras\src\engine\training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "C:\InstalledUtilities\Anaconda\lib\site-packages\keras\src\engine\training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\InstalledUtilities\Anaconda\lib\site-packages\keras\src\engine\training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "C:\InstalledUtilities\Anaconda\lib\site-packages\keras\src\engine\training.py", line 2381, in predict_step
        return self(x, training=False)
    File "C:\InstalledUtilities\Anaconda\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\InstalledUtilities\Anaconda\lib\site-packages\keras\src\engine\input_spec.py", line 298, in assert_input_